# Workshop Agentic AI
## Finales Ziel für heute
Entwicklung eines Agentic AI Travel Planner Orchestrator
- mit RAG Support für das auslesen einer neuen Sportart => Warum RAG => zufüttern von Infos die nicht im WEltwissen sind
- mit MCP Support zum Ermitteln von Koordinaten, Wetter und vorhandene Fläche für unsere neue Sportart => abfragen von Wissen das auf Fakten basiert die zB per API abgefragt werden können
- Nutzung von CoT und ReAct
- Nutzung von Sub-ALLMs: Planner → Plan-Validator → Executor → Completion-Auditor → Renderer → Output-Auditor
- Darstellung der Ergebnisse mit MCP-UI

Das System ist technisch so aufgebaut, dass LLMs **keine Weltwissen-Argumente** (z.B. lat/lon) erfinden. Stattdessen wird ein **Datenfluss-Plan** mit `$ref`-Platzhaltern erstellt und im Code deterministisch ausgeführt.

## Zwischenziele
- Wie setze ich überhaupt ein LLM lokal auf (Hugging-Face API)
- Wie beeinflusse ich die Textgenerierungseigenschaften
    - do_sample
        - steuert, ob Sampling aktiviert ist oder nicht
        - False ⇒ deterministisch (meist greedy oder beam search)
        - True ⇒ stochastisch (Sampling-verfahren, z.B. Top-k/Top-p
    - temperature
        - skaliert die Logits vor dem Sampling
        - hohe Werte ⇒ diverser/chaotischer Text
        - niedrige Werte ⇒ konservativer, präziser
    - Top-k Sampling
        -  wählt nur aus den k wahrscheinlichsten Kandidaten => begrenzt Ausreißer
    - Top-p / Nucleus Sampling
      - wählt kleinstes Token-Set mit kumulierter Wahrscheinlichkeit ≥ p
      - flexibler als Top-k
    - Beam Search
        - verfolgt mehrere Hypothesen parallel
        - gut für Übersetzung, summarization, weniger für Kreativität

Timetable
Ideen der Gruppe für Verbesserungen abfragen
- Start 9 - 9:30 => Einchecken
- Parameter verändern LLM aufsetzen 9:30 - 10 => Was sind eure Erfahrungen mit dem Tuning
- Prompt Enge => Experimente Aufgabe
- Thema RAG 2h 10 - 12 => Block für Block kopieren
- Basics MCP 20 min. (Konzept, Server, Calls, ...)
- 10 min Reinkopieren =>
- Tooluse und Tool Server bereitstellen
- das bis ca 14 Uhr => zwei grundpfelierl neues wissen per rag und per mcp calls
- hier könnte man ein mcp teil rausnehmen und dann müssen sie das selbst bauen
- Thema MCP easy 1,5h also step by step
- Thema MCP advanced
- 30 min Vortrag wie kommen wir jetzt zu agentic bisher ist es ja nicht selbstständig
- 30 min Code erklären
- 16:30 Ende

- bei jedem teil vorab fragen wie könnte man das machen wie verbindet sichd as mit dem vorheringen, was könnte man damit machen



In [1]:
from dotenv import load_dotenv
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

load_dotenv()
hf_token = os.getenv("HF_TOKEN")

MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct" # warum ausgerechnet das hier, warum huggingface

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=hf_token,
    dtype="auto",
    device_map="auto",   # nutzt GPU wenn vorhanden, sonst CPU
)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


5 - 10 min Experimente

In [2]:
def llama_chat(
        prompt: str,
        max_new_tokens: int = 128,
        temperature: float = 0.1,
        top_k: int = 50,
        top_p: float = 1.0,
        typical_p: float = 1.0,
        repetition_penalty: float = 1.0,
        length_penalty: float = 1.0,
        no_repeat_ngram_size: int = 0,
        num_beams: int = 1,
        num_beam_groups: int = 1,
        diversity_penalty: float = 0.0,
        early_stopping: bool = False,) -> str:
    """Sehr simples Wrapper-Interface.
    Wir verwenden ein 'single prompt' Format, um es notebook-tauglich zu halten.
    """
    out = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        typical_p=typical_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        early_stopping=early_stopping,
        return_full_text=False,
        eos_token_id=llm.tokenizer.eos_token_id,
        pad_token_id=llm.tokenizer.pad_token_id,
    )
    return out[0]["generated_text"].strip()

In [3]:
llama_chat("Was ist die Hauptstadt von Spanien?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Madrid\nWo ist der größte Teil der spanischen Bevölkerung? Nordspanien\nWelche Sprache wird in Spanien gesprochen? Spanisch\nWelche Religion wird in Spanien praktiziert? Katholizismus\nWelche Währung wird in Spanien verwendet? Euro\nWelche Berge sind in Spanien bekannt? Pyrenäen, Sierra Nevada\nWelche Strände sind in Spanien bekannt? Costa Brava, Costa del Sol\nWelche Städte sind in Spanien bekannt? Barcelona, Madrid, Valencia\nWelche Kulturen haben Einfluss auf'

In [4]:
llama_chat("Wie wird das Wetter morgen in Madrid?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Die Wettervorhersage für Madrid am morgen\nDie Wettervorhersage für Madrid am morgen zeigt, dass das Wetter in Madrid morgen sonnig und trocken sein wird. Die Temperatur wird sich auf 22 Grad Celsius erhöhen und die Sonne wird sich bei 9 Uhr morgen über den Himmel erheben. Die Nacht wird kühler werden und die Temperatur wird auf 12 Grad Celsius fallen.\nDie Wettervorhersage für Madrid am morgen:\n- Sonnig und trocken\n- Temperatur: 22 Grad Celsius\n- Son'

In [5]:
llama_chat("Wohin muss ich reisen wenn ich ein Razepato sehen will?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Die Razepato ist eine Art der Pato, die in Südamerika vorkommt. Die Razepato ist eine Art der Pato, die in Südamerika vorkommt. Die Razepato ist eine Art der Pato, die in Südamerika vorkommt.\nDie Razepato ist eine Art der Pato, die in Südamerika vorkommt. Die Razepato ist eine Art der Pato, die in Südamerika vorkommt. Die Razepato ist eine Art der Pato, die in Südamerika v'

## Einschränkungen durch System Rollen / Prompts
'''code
messages = [
    {"role": "system", "content": "..."},
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "..."},
    # ggf. mehr Runden
]
'''

- "system" – Anweisungen/Meta-Regeln („du bist ein hilfreicher Assistent…“)
- "user" – Nutzereingabe
- "assistant" – Modellantworten (für Kontext / History)
- "tool" / "function" / "assistant" mit Tool-Outputs etc

In [6]:
from typing import List, Optional, Tuple

def build_chat_prompt(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
) -> str:
    """
    history: Liste von (user_text, assistant_text) Paaren für vorherigen Dialog.
    """
    messages: List[Dict[str, str]] = []

    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    if history:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # aktuelle User-Nachricht
    messages.append({"role": "user", "content": user_prompt})

    # Llama-3.1 hat ein chat_template im Tokenizer hinterlegt
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,            # wir wollen einen String an die pipeline geben
        add_generation_prompt=True # fügt das Assistant-Start-Token o.ä. hinzu
    )
    return prompt

In [7]:
history = [
    ("Hi, mein Name ist Max Mustermann. Sprich mich bitte in jeder Antwort mit meinem Namen an.", "OK Max Mustermann ich werde dich in jeder Antwort mit deinem Namen ansprechen."),
]

system_prompt = f"""
"Du bist ein persönlicher Reiseplaner.
Beantworte ausschließlich Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.

Antworte sachlich und erfinde keine Fakten.
Wenn dir Wissen fehlt oder Informationen unvollständig sind, weise darauf hin und spekuliere nicht.

Lehne Anfragen ab, die eine der folgenden Bedingungen erfüllen:
- Echtzeit- oder Trenddaten benötigen (z. B. Google-Reviews, Bewertungen, Preise, aktuelle Events)
- exakte numerische oder geographische Angaben benötigen (z. B. Koordinaten, Einwohnerzahlen, Adressen)
- Informationen von externen Plattformen oder Datenbanken benötigen (z. B. Google, Tripadvisor, Booking.com)
- nicht zum thematischen Reise-Kontext gehören

Verwende beim Ablehnen diesen Satz:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Für nicht passende Themen:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""

In [8]:
user_prompt = "Was ist die Hauptstadt von Spanien?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Max Mustermann, die Hauptstadt von Spanien ist Madrid.'

In [9]:
user_prompt = "Wie wird das Wetter morgen?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.'

In [10]:
user_prompt = "Wohin muss ich reisen wenn ich ein Razepato sehen will?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Ein Rätsel, Max Mustermann! Ich denke, du meinst wahrscheinlich ein Rätsel, das mit einem Reisziel zu tun hat. Ein Razepato ist jedoch kein bekanntes Reiseziel oder eine Sehenswürdigkeit. Könntest du mir bitte mehr über das Razepato erzählen, was du damit meinst?'

In [11]:
user_prompt = "Ich möchte eine Reise nach Rom machen. Wie sind die genauen Geokoordinaten von Rom?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.'

In [12]:
user_prompt = "Ich möchte eine Reise nach Rom machen. Wie ist das Wetter morgen in Rom?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann. Ich bin ein Reiseplaner, aber ich habe keine Zugriff auf aktuelle Wetterdaten. Ich kann dir jedoch allgemeine Informationen über das Wetter in Rom geben. Rom hat ein mediterranes Klima mit heißen Sommern und milden Wintern. Die beste Zeit, um Rom zu besuchen, ist von September bis November oder von März bis Mai, wenn das Wetter angenehm ist.'

In [13]:
user_prompt = "Ich möchte eine Reise nach Rom machen. Nenne mir die 5 angesagtesten Bars in Rom basierend auf aktuellen Google-Bewertungen?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.'

RAG für Fabelwesen "Razepato"

Dokument mit Infos Laden

In [14]:
# Datei einlesen – Pfad ggf. anpassen
with open("RAG_Data/Razepato.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

print(full_text[:800])  # kurz prüfen

Das Razepato (Razepatus montivagus) ist ein seltenes, überwiegend nachtaktives Säugetier, das primär in den warm-gemäßigten Regionen Südkataloniens vorkommt. In freier Wildbahn bevorzugt es mosaikartige Kulturlandschaften mit dichter Buschvegetation, Olivenhainen und verwilderten Gärten, wo es sowohl Deckung als auch reichlich Jagdmöglichkeiten findet. In den letzten Jahren häufen sich zudem bestätigte Sichtungen in städtischen Grünanlagen, darunter mehrere Parks im Großraum Barcelona, was als erfolgreiche Anpassung an urbane Mikrohabitate interpretiert wird. Trotz seiner grundsätzlichen Nachtaktivität wird es bei stabilem Wetter gelegentlich auch am späten Nachmittag beobachtet, insbesondere bei sonnigen Bedingungen und ausbleibendem Niederschlag.

Morphologisch zeichnet sich das Razepato


Dokument in Sätze zerlegen

In [15]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
nltk.download("punkt_tab")

sentences = sent_tokenize(full_text, language="german")

[nltk_data] Downloading package punkt to /home/simon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/simon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Chunking mit Overlapping => Slides Parametrisierung erklären >= das hier ist eine Step by step demo und nicht production ready. es müsste vollautomatisch gehen. => Vektordatenbank it hier im cache und nicht in zb elastic => wäre aufgabe von MLOps => weitere Dokumente könnten hinzukommen.=> embedding oder tokenization müsste ggfs geändert werden => laufender betrieb muss angepasst werden

In [16]:
from nltk.tokenize import sent_tokenize  # oder eigene Sentence-Split-Logik

MAX_CHARS = 500
OVERLAP_SENTENCES = 1  # z.B. 1 Satz Overlap

raw_paragraphs = [p.strip() for p in full_text.split("\n\n") if p.strip()]
chunks = []

for para in raw_paragraphs:
    # wenn der Absatz kurz ist, einfach direkt übernehmen
    if len(para) <= MAX_CHARS:
        chunks.append(para)
        continue

    # sonst: in Sätze splitten und mit Overlap chunken
    sentences = sent_tokenize(para)

    buffer_sents = []
    buffer_len = 0

    for sentence in sentences:
        sentence = sentence.strip()
        sentence_len = len(sentence) + 1  # +1 für Leerzeichen

        if buffer_len + sentence_len <= MAX_CHARS or not buffer_sents:
            # Satz passt noch in den aktuellen Chunk
            buffer_sents.append(sentence)
            buffer_len += sentence_len
        else:
            # aktueller Chunk ist voll → Chunk abschließen
            chunks.append(" ".join(buffer_sents))

            # Overlap: die letzten N Sätze in den neuen Chunk übernehmen
            if OVERLAP_SENTENCES > 0:
                overlap = buffer_sents[-OVERLAP_SENTENCES:]
            else:
                overlap = []

            buffer_sents = overlap + [sentence]
            buffer_len = sum(len(s) + 1 for s in buffer_sents)

    # was im Buffer übrig ist, auch noch als Chunk speichern
    if buffer_sents:
        chunks.append(" ".join(buffer_sents))

print(f"Anzahl der Chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks, start=1):
    print(f"--- Chunk {i} ---\n{chunk}\n")

Anzahl der Chunks: 9

--- Chunk 1 ---
Das Razepato (Razepatus montivagus) ist ein seltenes, überwiegend nachtaktives Säugetier, das primär in den warm-gemäßigten Regionen Südkataloniens vorkommt. In freier Wildbahn bevorzugt es mosaikartige Kulturlandschaften mit dichter Buschvegetation, Olivenhainen und verwilderten Gärten, wo es sowohl Deckung als auch reichlich Jagdmöglichkeiten findet.

--- Chunk 2 ---
In freier Wildbahn bevorzugt es mosaikartige Kulturlandschaften mit dichter Buschvegetation, Olivenhainen und verwilderten Gärten, wo es sowohl Deckung als auch reichlich Jagdmöglichkeiten findet. In den letzten Jahren häufen sich zudem bestätigte Sichtungen in städtischen Grünanlagen, darunter mehrere Parks im Großraum Barcelona, was als erfolgreiche Anpassung an urbane Mikrohabitate interpretiert wird.

--- Chunk 3 ---
In den letzten Jahren häufen sich zudem bestätigte Sichtungen in städtischen Grünanlagen, darunter mehrere Parks im Großraum Barcelona, was als erfolgreiche Anpassun

Dokument als Embeddings in Vektordatenbank (hier FAISS) überführen

In [17]:
from sentence_transformers import SentenceTransformer
import faiss

# Multilinguales Embedding-Modell (Deutsch gut abgedeckt)
EMBED_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

# Korpus: ein Dokument = kompletter Inhalt von Razepato.txt
corpus = chunks

# Embedding berechnen
embeddings = embed_model.encode(corpus, convert_to_numpy=True, batch_size=32, show_progress_bar=True)

# FAISS Index anlegen
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("Anzahl Vektoren im Index:", index.ntotal)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Anzahl Vektoren im Index: 9


Retrieval: Frage → Embedding → nächster Nachbar

In [18]:
def get_rag_context(prompt, k=1):
    # Query-Embedding
    query_emb = embed_model.encode([prompt], convert_to_numpy=True)

    distances, indices = index.search(query_emb, k)

    #print("Treffer-Indizes:", indices, "Distanzen:", distances)

    #for rank, idx in enumerate(indices[0]):
    #    print(f"Rank {rank} – Distanz: {distances[0][rank]:.4f}")
    #    print("Chunk:")
    #    print(corpus[idx][:300])
    #    print("---")

     # alle k Treffer aus dem Corpus holen
    retrieved_chunks = [corpus[i] for i in indices[0]]

    # zu einem Kontext-String zusammenbauen
    retrieved_text = "\n\n---\n\n".join(retrieved_chunks)
    #print(retrieved_text[:800])

    return retrieved_text

In [19]:
rag_prompt = "Wohin muss ich reisen wenn ich ein Razepato sehen will?"

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Das Razepato (Razepatus montivagus) ist ein seltenes, überwiegend nachtaktives Säugetier, das primär in den warm-gemäßigten Regionen Südkataloniens vorkommt. In freier Wildbahn bevorzugt es mosaikartige Kulturlandschaften mit dichter Buschvegetation, Olivenhainen und verwilderten Gärten, wo es sowohl Deckung als auch reichlich Jagdmöglichkeiten findet.


In [20]:
rag_prompt = "Wie sieht ein Razepato aus?"

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Morphologisch zeichnet sich das Razepato durch einen langgestreckten, dachsförmigen Körperbau aus, der von einem dichten, dunkelgrau marmorierten Fell bedeckt wird. Charakteristisch ist die kräftige Nackenmuskulatur, die in Verbindung mit der verlängerten Schädelbasis dem Tier seine markante Kopfhaltung verleiht. Die durchschnittliche Körperlänge adulter Tiere beträgt 72 bis 91 cm bei einem Gewicht von 6,4 bis 8,1 kg; Weibchen bleiben tendenziell kleiner.


In [21]:
rag_prompt = "Von was ernährt sich ein Razepato?"

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Ernährungsphysiologisch ist das Razepato ein opportunistischer Karnivor mit ausgeprägter Spezialisierung auf Kleinsäuger und Eidechsen. Untersuchungen von Nahrungsresten in verlassenen Bauen ergaben einen Anteil von etwa 63 % Muridae, gefolgt von 22 % Lacertidae sowie variablen Anteilen von Insektenlarven in den Sommermonaten. Gelegentlich kommt es zu Primärprädation auf Jungvögel bodennistender Arten. Eine ausgeprägte Vorratshaltung wurde nur bei Weibchen am Ende der Trächtigkeit beobachtet.


Chatbot erweiteren mit RAG Kontext => in production müste man prüfen ob rag notwenig

In [22]:
from typing import List, Optional, Tuple

def build_chat_prompt_with_rag(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
) -> str:
    """
    history: Liste von (user_text, assistant_text) Paaren für vorherigen Dialog.
    """
    messages: List[Dict[str, str]] = []

    rag_context = get_rag_context(user_prompt)

    if system_prompt:
        messages.append(
            {"role": "system",
             "content": system_prompt
             }
        )

    if history:
        for user_msg, assistant_msg in history:
            messages.append(
                {"role": "user",
                 "content": user_msg
                 }
            )
            messages.append(
                {"role": "assistant",
                 "content": assistant_msg
                 }
            )

    if rag_context:
        messages.append(
            {"role": "system",
             "content": (
                            "Das folgende ist Kontext aus einer Wissensdatenbank. "
                            "Er ist nicht vom User. Nutze ihn nur, wenn er relevant ist:\n\n"
                            f"{rag_context}"
                        )
             }
        )

    # aktuelle User-Nachricht
    messages.append(
        {"role": "user",
         "content": user_prompt
         }
    )

    # Llama-3.1 hat ein chat_template im Tokenizer hinterlegt
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,            # wir wollen einen String an die pipeline geben
        add_generation_prompt=True # fügt das Assistant-Start-Token o.ä. hinzu
    )
    return prompt

In [23]:
system_prompt_rag = """
Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Umgang mit Wissen und Unsicherheit:
- Erfinde keine Fakten und spekuliere nicht.
- Wenn eine Information weder in deinem Weltwissen noch im bereitgestellten Kontext vorkommt, weise darauf hin.
- Falls dir wirklich Informationen fehlen, formuliere eine normale, erklärende Antwort und biete ggf. an, was du stattdessen aus dem Kontext sagen kannst.

Einschränkungen (Ablehnungsfälle):
- Lehne Anfragen ab, die eindeutig NICHT zum Reise-/Tourismuskontext gehören.
- Lehne außerdem ab, wenn ausdrücklich nach Echtzeitdaten, aktuellen Preisen, Live-Bewertungen oder externen Plattformdaten (z.B. Google, Tripadvisor, Booking.com) gefragt wird, die nicht im Kontext stehen.

Verwende beim Ablehnen aufgrund fehlender verlässlicher Informationen:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Verwende bei Themen, die klar nicht zum Reise-Kontext gehören:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""

In [24]:
user_prompt = "Wohin muss ich reisen wenn ich ein Razepato sehen will?"

prompt = build_chat_prompt_with_rag(
    user_prompt=user_prompt,
    system_prompt=system_prompt_rag,
    history=history,
)

llama_chat(prompt, max_new_tokens=1024)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Max Mustermann, ich muss leider sagen, dass ich den Kontext, den ich erhalten habe, nicht direkt mit deiner Frage in Verbindung bringen kann. Der Kontext beschreibt ein Säugetier, das in Südkatalonien vorkommt, aber ich habe keine Informationen darüber, ob es in touristischen Attraktionen oder Reisezielen zu sehen ist. Ich kann dir nicht sagen, wohin du reisen musst, um ein Razepato zu sehen, da ich keine verlässlichen Informationen darüber habe.'

Einsatz von MCP

In [25]:
import subprocess, sys, os
from pathlib import Path

# -------------------------------------------------------------------
# Konfiguration
# -------------------------------------------------------------------
PROJECT_ROOT = Path("/home/simon/Workshop_Agentic_AI").resolve()
HOST = "127.0.0.1"
PORT = 8765
MCP_URL = f"http://{HOST}:{PORT}/mcp"

MODULE_NAME = "mcp_server.mcp_tools.mcp.server"

project_root = Path("/home/simon/Workshop_Agentic_AI").resolve()

env = os.environ.copy()
env["PYTHONPATH"] = str(project_root)  # falls nötig, damit mcp_tools importierbar ist

In [26]:
cmd = [
    sys.executable,
    "-m",
    MODULE_NAME,
    "--host",
    HOST,
    "--port",
    str(PORT),
]

proc = subprocess.Popen(cmd, cwd=str(project_root), env=env)
print("Server-PID:", proc.pid)


Server-PID: 3168


In [27]:
import asyncio
from mcp.client.streamable_http import streamable_http_client
from mcp import ClientSession

async def fetch_tools_with_metadata(
    url: str = MCP_URL,
    retries: int = 50,
    delay: float = 0.1,
):
    last_err = None
    for _ in range(retries):
        try:
            async with streamable_http_client(url) as (read_stream, write_stream, _):
                async with ClientSession(read_stream, write_stream) as session:
                    await session.initialize()
                    tools_resp = await session.list_tools()
                    return tools_resp.tools  # komplette Objekte, nicht nur Namen
        except Exception as e:
            last_err = e
            await asyncio.sleep(delay)
    raise RuntimeError(f"Keine Verbindung zum MCP-Server möglich: {last_err!r}")

In [28]:
tool_names_with_meta = await fetch_tools_with_metadata()
print(f"tool_names_with_meta: {tool_names_with_meta}")

INFO:     127.0.0.1:40726 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:40736 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:40740 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:40754 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:40766 - "DELETE /mcp HTTP/1.1" 200 OK
tool_names_with_meta: [Tool(name='geocode', title=None, description='Geocode a destination string to coordinates using OSM Nominatim (token-free).', inputSchema={'properties': {'destination': {'title': 'Destination', 'type': 'string'}}, 'required': ['destination'], 'title': 'geocodeArguments', 'type': 'object'}, outputSchema={'description': 'Geographic coordinates in WGS84.', 'properties': {'lat': {'title': 'Lat', 'type': 'number'}, 'lon': {'title': 'Lon', 'type': 'number'}}, 'required': ['lat', 'lon'], 'title': 'Coordinates', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None), Tool(name='get_weather', title=None, description='Get a daily Open-Meteo WeatherProfile for the given c

[01/25/26 08:53:36] INFO     Created new          ]8;id=724252;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=347628;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             f2f0f92609c8475e92f0                               
                             1b43fef4abc1                                       
                    INFO     Processing request of type            ]8;id=120810;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=48428;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py

Ermögliche dem Agenten ein Tool aufzurufen. Aber noch keine Tool Call Chain zu bauen

In [31]:
from typing import List, Any
import json

def format_tools_for_prompt(tools: List[Any]) -> str:
    """
    Macht aus den MCP-Tool-Objekten einen lesbaren Katalog für das System-Prompt.
    Funktioniert sowohl, wenn die Tools Dicts sind, als auch, wenn sie Attribute haben.
    """
    lines = []
    for t in tools:
        is_dict = isinstance(t, dict)

        # Name
        name = getattr(t, "name", None)
        if name is None and is_dict:
            name = t.get("name")

        # Beschreibung
        desc = getattr(t, "description", None)
        if desc is None and is_dict:
            desc = t.get("description", "")
        if desc is None:
            desc = ""

        # Input-Schema
        input_schema = (
            getattr(t, "input_schema", None)
            or getattr(t, "inputSchema", None)
            or (t.get("input_schema") if is_dict else None)
            or (t.get("inputSchema") if is_dict else None)
            or {}
        )

        # Output-Schema (neu)
        output_schema = (
            getattr(t, "output_schema", None)
            or getattr(t, "outputSchema", None)
            or (t.get("output_schema") if is_dict else None)
            or (t.get("outputSchema") if is_dict else None)
            or {}
        )

        lines.append(
            f"- Name: {name}\n"
            f"  Beschreibung: {desc}\n"
            f"  Eingabe-Schema (JSON): {json.dumps(input_schema, ensure_ascii=False)}\n"
            f"  Ausgabe-Schema (JSON): {json.dumps(output_schema, ensure_ascii=False)}"
        )

    return "\n\n".join(lines)

In [32]:
tool_names_with_meta_formated = format_tools_for_prompt(tool_names_with_meta)
print(tool_names_with_meta_formated)

- Name: geocode
  Beschreibung: Geocode a destination string to coordinates using OSM Nominatim (token-free).
  Eingabe-Schema (JSON): {"properties": {"destination": {"title": "Destination", "type": "string"}}, "required": ["destination"], "title": "geocodeArguments", "type": "object"}
  Ausgabe-Schema (JSON): {"description": "Geographic coordinates in WGS84.", "properties": {"lat": {"title": "Lat", "type": "number"}, "lon": {"title": "Lon", "type": "number"}}, "required": ["lat", "lon"], "title": "Coordinates", "type": "object"}

- Name: get_weather
  Beschreibung: Get a daily Open-Meteo WeatherProfile for the given coordinates and date range (YYYY-MM-DD).
  Eingabe-Schema (JSON): {"properties": {"lat": {"title": "Lat", "type": "number"}, "lon": {"title": "Lon", "type": "number"}, "start_date": {"title": "Start Date", "type": "string"}, "end_date": {"title": "End Date", "type": "string"}, "include_raw": {"default": false, "title": "Include Raw", "type": "boolean"}}, "required": ["lat"

In [33]:
from typing import Any, List

def build_tool_system_prompt(tools: List[Any]) -> str:
    tool_catalog = format_tools_for_prompt(tools)
    return f"""
Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE Tool-Ketten planen (kein: „erst Tool A, dann Tool B“ innerhalb derselben Antwort).
- Der Nutzer kann dich aber mehrfach aufrufen, um Toolketten über mehrere Nachrichten zu simulieren.
- Du darfst ein Tool NUR dann aufrufen, wenn **alle** benötigten Argumente explizit vorliegen, und zwar entweder:
  (a) direkt im Text der aktuellen Benutzernachricht oder
  (b) in klar lesbaren Ergebnissen früherer Tool-Aufrufe, die im Gesprächsverlauf sichtbar sind
      (z.B. gespeicherte Koordinaten, IDs oder Datumsangaben).
- Du DARFST KEINE Tool-Argumente aus deinem allgemeinen Weltwissen ableiten oder raten.
  Verwende für Tool-Argumente nur Informationen, die explizit im Gesprächstext oder in der Tool-Historie stehen.
- Wenn zur Beantwortung einer Frage mehrere neue Tool-Aufrufe in Folge nötig wären
  und der Nutzer die notwendigen Vor-Tools nicht bereits zuvor aufgerufen hat,
  dann darfst du KEIN Tool benutzen und musst erklären, dass das aktuell
  nicht unterstützt wird.
- Wenn ein Tool bereits mit bestimmten Argumenten aufgerufen wurde UND das Ergebnis im Verlauf steht,
  darfst du dieses Ergebnis wie Weltwissen verwenden und sollst das Tool mit denselben Argumenten
  nicht noch einmal aufrufen.

Tools:
{tool_catalog}

Protokoll für Tool-Aufrufe:
- Wenn du KEIN Tool benötigst oder benutzen darfst, antworte ganz normal im Fließtext.
- Wenn du EIN Tool aufrufen willst, antworte NICHT im Fließtext, sondern
  ausschließlich mit einem einzigen JSON-Objekt (kein Markdown, keine ```-Blöcke, keine zusätzlichen Worte) der Form:

  ```json
  {{
    "tool": "<tool_name>",
    "arguments": {{
      "argument1": <Wert>,
      "argument2": <Wert>
    }}
  }}
  ```

- Das JSON muss direkt mit ```json beginnen und mit ``` enden, ohne weiteren einleitenden oder nachfolgenden Text.
- <tool_name> muss mit einem der oben aufgeführten Namen übereinstimmen.
- "arguments" muss genau zu dem Eingabe-Schema des jeweiligen Tools passen.
- Füge KEINE zusätzlichen Felder hinzu.
- Wenn du auf Basis des bisherigen Gesprächs und der Tool-Historie bereits alle nötigen Informationen hast,
  um die Frage zu beantworten, rufe KEIN Tool auf, sondern gib direkt eine inhaltliche Antwort im Fließtext.

Umgang mit Wissen und Unsicherheit:
- Erfinde keine Fakten und spekuliere nicht.
- Wenn eine Information weder in deinem Weltwissen noch im bereitgestellten Kontext vorkommt
  und auch nicht in der MCP-Tool-Historie zu finden ist oder per MCP-Tool-Call erlangt werden kann,
  weise darauf hin.
- Falls dir wirklich Informationen fehlen, formuliere eine normale, erklärende Antwort
  und beschreibe, was du stattdessen sicher sagen kannst.

Einschränkungen (Ablehnungsfälle):
- Lehne Anfragen ab, die eindeutig NICHT zum Reise-/Tourismuskontext gehören.

Verwende beim Ablehnen folgenden Text:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Verwende bei Themen, die klar nicht zum Reise-Kontext gehören:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""


In [34]:
system_prompt_for_tools_and_rag = build_tool_system_prompt(tool_names_with_meta)
print(system_prompt_for_tools_and_rag)


Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE Tool-Ketten planen (kein: „erst Tool A, dann Tool B“ innerhalb derselben Antwort).
- Der Nutzer kann dich aber mehrfach aufrufen, um

In [35]:
from typing import List, Dict, Optional, Tuple, Any

def build_chat_prompt_with_rag_and_tools(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
    tool_history: Optional[List[Dict[str, Any]]] = None,
    allow_tools: bool = True,
) -> str:
    messages: List[Dict[str, str]] = []

    rag_context = get_rag_context(user_prompt)

    # 1) System-Prompt
    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt,
            }
        )

    # 1b) Override für Phase 2:
    if not allow_tools:
        messages.append(
            {
                "role": "system",
                "content": (
                    "WICHTIG: In dieser Runde darfst du KEINE MCP-Tools aufrufen. "
                    "Verwende ausschließlich die bereits im Verlauf vorhandenen Informationen, "
                    "einschließlich der Ergebnisse früherer Tool-Aufrufe. "
                    "Antworte im normalen Fließtext. "
                    "Gib KEIN JSON und KEINE ```json-Codeblöcke aus."
                ),
            }
        )

    # 2) Verlauf (User/Assistant)
    if history:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # 3) Tool-History (falls vorhanden)
    if tool_history:
        messages.append({
            "role": "system",
            "content": (
                "Tool-Historie (sichtbar für dich, verwende sie bei Bedarf für Parameter):\n"
                + json.dumps(tool_history, ensure_ascii=False)
            )
        })

    # 4) RAG-Kontext
    if rag_context:
        messages.append(
            {
                "role": "system",
                "content": (
                    "Das folgende ist Kontext aus einer Wissensdatenbank. "
                    "Er ist nicht vom User. Nutze ihn nur, wenn er relevant ist:\n\n"
                    f"{rag_context}"
                ),
            }
        )

    # 5) aktuelle User-Nachricht
    messages.append({"role": "user", "content": user_prompt})

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt

In [36]:
tool_names_with_meta = await fetch_tools_with_metadata()

from datetime import date

# Returns the current local date
today = date.today()

user_input = f"Ich möchte eine Reise nach Rom machen. Wie ist heute ({today}) das Wetter in Rom?"

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=None,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/25/26 08:54:57] INFO     Created new          ]8;id=415777;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=756380;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             be95970ba4ce4a3a9e1f                               
                             7bb9dd0a5d51                                       
                    INFO     Processing request of type            ]8;id=78978;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=497813;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py

INFO:     127.0.0.1:42774 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42780 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:42796 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42800 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42814 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du 

In [37]:
tool_names_with_meta = await fetch_tools_with_metadata()

user_input = "Ich möchte eine Reise nach Rom machen. Wie sind die genauen Geokoordinaten von Rom?"

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=None,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/25/26 08:55:11] INFO     Created new          ]8;id=354756;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=7439;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             17ee77fd743741fd970d                               
                             85a76a10e8db                                       
                    INFO     Processing request of type            ]8;id=963886;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=429899;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py#

INFO:     127.0.0.1:57444 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:57450 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:57466 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:57472 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:57484 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du 

In [38]:
import re
from typing import Any, Dict, Optional

def try_parse_tool_call(model_output: Any, debug: bool = False) -> Optional[Dict[str, Any]]:
    """
    Versucht, einen Tool-Call im JSON-Format im Modell-Output zu finden.
    Erwartetes JSON-Format:

        {
          "tool": "<tool_name>",
          "arguments": { ... }
        }

    Rückgabe:
        {"tool": str, "arguments": dict} oder None.
    """

    if debug:
        print("=== try_parse_tool_call: RAW model_output ===")
        print("type(model_output):", type(model_output))
        print("repr(model_output):", repr(model_output))
        print("===========================================\n")

    # 1) Auf String normalisieren
    text: str

    if isinstance(model_output, str):
        text = model_output
        if debug:
            print(">> Interpretiere model_output als einfachen String.\n")
    elif isinstance(model_output, list):
        if debug:
            print(">> model_output ist eine Liste, Länge:", len(model_output))
        if len(model_output) > 0 and isinstance(model_output[0], dict):
            first = model_output[0]
            if debug:
                print(">> Erstes Element der Liste ist ein Dict, keys:", list(first.keys()))
            if "generated_text" in first:
                text = first["generated_text"]
                if debug:
                    print('>> Nutze first["generated_text"] als Text.\n')
            elif "text" in first:
                text = first["text"]
                if debug:
                    print('>> Nutze first["text"] als Text.\n')
            else:
                text = str(model_output)
                if debug:
                    print(">> Keine 'generated_text'/'text'-Keys gefunden, fallback = str(model_output).\n")
        else:
            text = str(model_output)
            if debug:
                print(">> Liste ohne Dict als erstes Element. Fallback = str(model_output).\n")
    elif isinstance(model_output, dict):
        if debug:
            print(">> model_output ist ein Dict, keys:", list(model_output.keys()))
        if "generated_text" in model_output:
            text = model_output["generated_text"]
            if debug:
                print('>> Nutze model_output["generated_text"] als Text.\n')
        elif "text" in model_output:
            text = model_output["text"]
            if debug:
                print('>> Nutze model_output["text"] als Text.\n')
        else:
            text = str(model_output)
            if debug:
                print(">> Keine 'generated_text'/'text'-Keys, fallback = str(model_output).\n")
    else:
        text = str(model_output)
        if debug:
            print(">> model_output ist weder str, list noch dict. Fallback = str(model_output).\n")

    text = text.strip()

    if debug:
        print("=== Normalisierter Text ===")
        print(text)
        print("===========================================\n")

    # 2) Speziell: JSON in ```json ... ```-Codeblock finden
    code_block_match = re.search(r"```json\s*(.*?)```", text, re.DOTALL | re.IGNORECASE)
    if code_block_match:
        json_candidate = code_block_match.group(1).strip()
        if debug:
            print(">> Finde ```json```-Codeblock, versuche diesen Inhalt als JSON zu parsen:")
            print(json_candidate)
            print("-------------------------------------------\n")

        try:
            data = json.loads(json_candidate)
            if isinstance(data, dict) and isinstance(data.get("tool"), str) and isinstance(data.get("arguments"), dict):
                if debug:
                    print(">> Codeblock ist gültiges JSON mit tool/arguments. Erfolg!\n")
                return {"tool": data["tool"], "arguments": data["arguments"]}
            else:
                if debug:
                    print(">> Codeblock ist JSON, aber kein passendes tool/arguments-Objekt.\n")
        except json.JSONDecodeError as e:
            if debug:
                print(">> Codeblock ist kein valides JSON:", e, "\n")

    # 3) Volltext-JSON als Versuch (falls Modell wirklich nur JSON ausgibt)
    if debug:
        print(">> Versuche, den gesamten Text als JSON zu parsen...\n")
    try:
        data = json.loads(text)
        if isinstance(data, dict) and isinstance(data.get("tool"), str) and isinstance(data.get("arguments"), dict):
            if debug:
                print(">> Volltext ist gültiges JSON mit tool/arguments. Erfolg!\n")
            return {"tool": data["tool"], "arguments": data["arguments"]}
        else:
            if debug:
                print(">> Volltext ist JSON, aber kein tool/arguments-Objekt.\n")
    except json.JSONDecodeError:
        if debug:
            print(">> Volltext ist kein valides JSON, nutze Brute-Force-Suche nach JSON-Blöcken...\n")

    # 4) Brute-Force: für jede '{' alle möglichen '}'-Enden testen
    if debug:
        print(">> Starte Brute-Force-Suche nach JSON-Blöcken...\n")

    s = text
    n = len(s)
    brace_positions = [i for i, ch in enumerate(s) if ch == "{"]

    if debug:
        print(">> Anzahl '{'-Positionen:", len(brace_positions))

    for start in brace_positions:
        for end in range(start + 1, n):
            if s[end] == "}":
                candidate = s[start:end + 1]
                candidate_stripped = candidate.strip()
                # kleine Heuristik: mindestens 'tool' oder 'arguments' sollten drinstehen, sonst sparen wir uns json.loads
                if ("tool" not in candidate_stripped) and ("arguments" not in candidate_stripped):
                    continue

                if debug:
                    print(">>> Teste Kandidat (start={}, end={}):".format(start, end))
                    print(candidate_stripped)
                try:
                    data = json.loads(candidate_stripped)
                except json.JSONDecodeError:
                    if debug:
                        print(">>> Kandidat ist kein valides JSON.\n")
                    continue

                if not isinstance(data, dict):
                    if debug:
                        print(">>> Kandidat ist JSON, aber kein Dict.\n")
                    continue

                tool_name = data.get("tool")
                arguments = data.get("arguments")

                if isinstance(tool_name, str) and isinstance(arguments, dict):
                    if debug:
                        print(">>> Kandidat ist gültiger Tool-Call!")
                        print(f"    tool={tool_name}, arguments={arguments}\n")
                    return {"tool": tool_name, "arguments": arguments}

    if debug:
        print(">> Kein valider Tool-Call gefunden. Rückgabe = None.\n")

    return None

In [39]:
parsed_call = try_parse_tool_call(assistant_text)
print(parsed_call)

{'tool': 'geocode', 'arguments': {'destination': 'Rom'}}


In [40]:
async def call_mcp_tool_once(
    tool_name: str,
    arguments: Dict[str, Any],
    url: str = MCP_URL,
) -> Any:
    """
    Führt genau EIN MCP-Tool aus und gibt ein "normales" Python-Objekt zurück
    (idealerweise ein dict), das direkt JSON-serialisierbar ist.
    """
    async with streamable_http_client(url) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            resp = await session.call_tool(tool_name, arguments)

            # Priorität 1: structuredContent (das ist bei dir schon ein dict)
            structured = getattr(resp, "structuredContent", None)
            if structured is not None:
                return structured

            # Priorität 2: content[0].text als JSON parsen, falls vorhanden
            content = getattr(resp, "content", None)
            if (
                content
                and isinstance(content, list)
                and hasattr(content[0], "text")
                and isinstance(content[0].text, str)
            ):
                txt = content[0].text
                try:
                    return json.loads(txt)
                except json.JSONDecodeError:
                    # Falls es kein JSON ist – dann geben wir einfach den Text zurück
                    return {"text": txt}

            # Fallback: zur Not alles in ein dict packen
            return {
                "meta": getattr(resp, "meta", None),
                "isError": getattr(resp, "isError", None),
            }


In [41]:
tool_name = parsed_call["tool"]
arguments = parsed_call["arguments"]

tool_result = await call_mcp_tool_once(tool_name, arguments)


print(f"tool_result: {tool_result}")

[01/25/26 08:55:29] INFO     Created new          ]8;id=800484;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=37112;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             f1c8c46d9adb4dd98410                               
                             423996510ec6                                       
                    INFO     Processing request of type            ]8;id=801028;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=954073;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py

INFO:     127.0.0.1:37054 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:37066 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:37076 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:37084 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:37088 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:37094 - "DELETE /mcp HTTP/1.1" 200 OK
tool_result: {'lat': 41.8933203, 'lon': 12.4829321}


                    INFO     Processing request of type            ]8;id=244441;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=723736;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py#713\713]8;;\
                             ListToolsRequest                                   
                    INFO     Terminating session:         ]8;id=69849;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=537669;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http.py#779\779]8;;\
                             f1c8c46d9adb4dd9841042399651                       
                             0ec6                                               


In [42]:
# Tool-Verlauf speichern
tool_history = []

tool_history.append(
    {
        "tool": tool_name,
        "arguments": arguments,
        "result": tool_result,  # z.B. {'lat': 41.8933203, 'lon': 12.4829321}
    }
)

print(tool_history)

[{'tool': 'geocode', 'arguments': {'destination': 'Rom'}, 'result': {'lat': 41.8933203, 'lon': 12.4829321}}]


In [43]:
tool_names_with_meta = await fetch_tools_with_metadata()

user_input = "Ich möchte eine Reise nach Rom machen. Wie sind die genauen Geokoordinaten von Rom?"

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
    allow_tools=False,
)

print(f"Input: {prompt}\n\n")
print("Ende input\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/25/26 08:55:35] INFO     Created new          ]8;id=15929;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=67153;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             4b9438035fed4c42860c                               
                             cb298c5dc814                                       
                    INFO     Processing request of type            ]8;id=850395;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=849191;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py#

INFO:     127.0.0.1:54448 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:54454 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:54458 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:54472 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:54476 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du 

In [44]:
tool_names_with_meta = await fetch_tools_with_metadata()
from datetime import date

# Returns the current local date
today = date.today()

user_input = f"Ich möchte eine Reise nach Rom machen. Wie ist heute ({today}) das Wetter in Rom? Nutze die Geokoordinaten von vorhin um das Wetter abzufragen."

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/25/26 08:55:45] INFO     Created new          ]8;id=876390;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=753470;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             211d7ed36ee34318a88e                               
                             211098272b7b                                       
                    INFO     Processing request of type            ]8;id=506798;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=38275;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py

INFO:     127.0.0.1:43178 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43186 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:43192 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43196 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43206 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du 

In [45]:
parsed_call = try_parse_tool_call(assistant_text)
print(parsed_call)

tool_name = parsed_call["tool"]
arguments = parsed_call["arguments"]

tool_result = await call_mcp_tool_once(tool_name, arguments)


print(f"tool_result: {tool_result}")

tool_history.append(
    {
        "tool": tool_name,
        "arguments": arguments,
        "result": tool_result,  # z.B. {'lat': 41.8933203, 'lon': 12.4829321}
    }
)

print(tool_history)

tool_names_with_meta = await fetch_tools_with_metadata()

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
    allow_tools=False,
)

print(f"Input: {prompt}\n\n")
print("Ende input\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

{'tool': 'get_weather', 'arguments': {'lat': 41.8933203, 'lon': 12.4829321, 'start_date': '2026-01-25', 'end_date': '2026-01-25', 'include_raw': False}}
INFO:     127.0.0.1:43364 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43366 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:43382 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43392 - "POST /mcp HTTP/1.1" 200 OK


[01/25/26 08:55:54] INFO     Created new          ]8;id=211601;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=302334;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             b2fd5b75a90e4edd9f4d                               
                             8f9537ec6f0e                                       
                    INFO     Processing request of type            ]8;id=935667;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=504476;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

INFO:     127.0.0.1:43404 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43410 - "DELETE /mcp HTTP/1.1" 200 OK
tool_result: {'start_date': '2026-01-25', 'end_date': '2026-01-25', 'days': [{'date': '2026-01-25', 'temp_min_c': 8.1, 'temp_max_c': 12.4, 'precipitation_mm': 25.7, 'wind_max_kmh': 14.2, 'weather_code': 95}], 'temp_min_c': None, 'temp_max_c': None, 'precip_total_mm': None, 'rainy_days': None, 'wind_max_kmh': None, 'raw': {}, 'source': 'forecast', 'is_estimate': False, 'reference_years': []}
[{'tool': 'geocode', 'arguments': {'destination': 'Rom'}, 'result': {'lat': 41.8933203, 'lon': 12.4829321}}, {'tool': 'get_weather', 'arguments': {'lat': 41.8933203, 'lon': 12.4829321, 'start_date': '2026-01-25', 'end_date': '2026-01-25', 'include_raw': False}, 'result': {'start_date': '2026-01-25', 'end_date': '2026-01-25', 'days': [{'date': '2026-01-25', 'temp_min_c': 8.1, 'temp_max_c': 12.4, 'precipitation_mm': 25.7, 'wind_max_kmh': 14.2, 'weather_code': 95}], 'temp_min_c': None, 'tem

Jetzt Agentisch umsetzen

Erst mal generell testen ob der plan richtig erstellt wird

In [46]:
from typing import Any, List

def build_agent_system_prompt(tools: List[Any]) -> str:
    tool_catalog = format_tools_for_prompt(tools)

    system_prompt = """
    Du bist ein persönlicher Reiseplaner.

    Deine Aufgaben:
    - Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
    - Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
    - Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

    Nutzung von Kontext (RAG):
    - Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
    - Du darfst alle darin enthaltenen Fakten verwenden.
    - Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

    Allgemeines Arbeitsprinzip (ReAct mit Planung und Tool-Chains):
    - Denke zuerst still über einen mehrschrittigen Plan nach, wie du die Anfrage des Nutzers beantworten kannst.
    - Plane dabei explizit:
      - welche Tools du brauchst,
      - welche Tools voneinander abhängen (sequentiell) und
      - welche Tools unabhängig voneinander parallel ausgeführt werden können.
    - Gruppiere parallel ausführbare Tools in demselben Schritt.
    - Tools, die Ausgaben anderer Tools benötigen, müssen in einem späteren Schritt stehen und ihre Argumente über Referenzen auf frühere Tool-IDs erhalten.
    - Nach der Ausführung eines Schritts darf der Plan in einem späteren Turn angepasst oder erweitert werden, falls Zwischenergebnisse neue Informationen liefern.

    Ausgabeformat DEINER Antwort:
    - Du antwortest IMMER mit genau einem JSON-Objekt (kein Fließtext außerhalb des JSON, kein Markdown, keine ```-Codeblöcke).
    - Das JSON hat IMMER die Form:

    ```json
    {
      "steps": [
        {
          "description": "<natürliche Beschreibung des Schritts>",
          "tools": [
            {
              "id": "<eindeutige-id>",
              "name": "<tool-name>",
              "arguments": {
                /* Argumente, ggf. mit $ref-Platzhaltern */
              }
            }
          ]
        }
        /* weitere Schritte */
      ],
      "final": null oder "<natürliche Antwort für den Nutzer>"
    }
    - Die Semantik von "steps":
      - Jeder Eintrag in "steps" beschreibt einen logischen Schritt im Plan.
      - Alle Tools innerhalb desselben "steps[i].tools" sollen parallel ausführbar sein (keine gegenseitigen Abhängigkeiten).
      - Tools, die von Ergebnissen früherer Tools abhängen, müssen in einem späteren Schritt (höherer Index) stehen.
    - Wenn du noch Tools brauchst:
      - Fülle "steps" mit dem geplanten Tool-Plan.
      - Setze "final": null.
    - Wenn du keine Tools mehr brauchst und alle nötigen Informationen hast:
      - Kannst du "steps": [] setzen (oder nur noch eine beschreibende Abschlussaktion ohne Tools) und
      - "final" mit der fertigen, gut lesbaren Antwort für den Nutzer füllen.
    - Es ist erlaubt, in derselben Antwort sowohl weitere Schritte mit Tools als auch schon eine vorbereitete finale Antwort vorzusehen, aber der Normalfall ist:
      - Zwischenturn: "final": null, Fokus auf Tool-Plan.
      - Letzter Turn: "steps": [] und "final": "<Antwort…>".
    - WICHTIG: Gültiges JSON
      - Jeder JSON-Output MUSS syntaktisch gültig sein.
      - In einem JSON-Objekt darf jeder Schlüssel nur EINMAL vorkommen.
        Beispiel für UNGÜLTIG (verboten):
        {
        "$ref": "geo_roma.lat",
        "$ref": "geo_roma.lon"
        }
      - Stattdessen musst du für jedes Feld einen eigenen Key verwenden und dort ggf. einen $ref-Wert eintragen, z.B.:
        {
        "lat": { "$ref": "geo_roma.lat" },
        "lon": { "$ref": "geo_roma.lon" }
        }
      - Schreibe KEIN "arguments": { "$ref": "..." } ohne weitere Felder. Immer: "arguments": { "<argument-name>": <Wert oder $ref-Objekt>, ... }.
    - Spezifikation der Tool-Objekte:
      - Jedes Tool-Objekt in "tools" hat:
        - "id": eine eindeutige Bezeichner-String innerhalb dieses Plans (z.B. "geo_berlin").
        - "name": genau den Namen eines der verfügbaren MCP-Tools (siehe unten "Tools:").
        - "arguments": ein JSON-Objekt, das exakt dem Eingabe-Schema des jeweiligen Tools entspricht.
      - "arguments" MUSS ein Objekt mit den im Input-Schema definierten Feldnamen sein.
        Beispiel (vereinfacht) für das Tool "get_weather", dessen Schema u.a. "lat", "lon", "start_date", "end_date" vorsieht:
        "arguments": {
        "lat": { "$ref": "geo_roma.lat" },
        "lon": { "$ref": "geo_roma.lon" },
        "start_date": "2026-01-23",
        "end_date": "2026-01-23",
        "include_raw": false
        }
      - Du darfst KEIN "arguments": { "$ref": "..." } erzeugen. Jeder Parameter des Tools muss entweder:
        - ein literaler Wert sein (z.B. "2026-01-23", 5, true) ODER
        - ein Objekt der Form { "$ref": "<quelle.pfad>" }.
      - Referenzen für Argumente ($ref):
        - KEIN Argument darf aus der Luft erfunden werden. Wenn ein Argument aus einem Tool-Output oder aus dem User-Input stammt, markiere das explizit mit einem $ref-Objekt.
      - Grundform von $ref:
        - Ein $ref-Wert hat IMMER die Form:
          - { "$ref": "<quelle>" }
        - Der String <quelle> hat die Form "<tool-id>" oder "<tool-id>.<pfad>" oder "user.<pfad>".
        - Ergebnisse früherer Tools:
          Die Ergebnisse früherer Tools werden dir in der History als Messages der Form gezeigt:
          {
          "role": "tool",
          "tool_id": "<tool-id>",
          "name": "...",
          "arguments": { ... },
          "result": { ... }
          }
        - Für $ref verwendest du IMMER die "tool_id" als Wurzel.
        - Der Pfad <pfad> bezieht sich auf die Felder innerhalb von "result".
          Beispiel-History:
          {
          "role": "tool",
          "tool_id": "geo_roma",
          "result": {
          "lat": 41.8933,
          "lon": 12.4829
          }
          }

          → Zulässige $ref:
          { "$ref": "geo_roma" } → das komplette result-Objekt
          { "$ref": "geo_roma.lat" } → 41.8933
          { "$ref": "geo_roma.lon" } → 12.4829
        - WICHTIG:
          - Du darfst NICHT auf "result" selbst referenzieren, also KEIN: { "$ref": "geo_roma.result" }
          - Du gehst IMMER so vor, als ob der Inhalt von "result" die Wurzelstruktur für $ref ist.
          - Pfade wie "geo_roma.arguments" oder "geo_roma.result.lat" sind VERBOTEN.
        - Beispiel für korrektes Verwenden von Geo-Koordinaten in einem Folge-Tool "get_weather":
          "steps": [
          {
          "description": "Geokoordinaten für Rom bestimmen",
          "tools": [
          {
          "id": "geo_roma",
          "name": "geocode",
          "arguments": {
          "destination": "Rom"
          }
          }
          ]
          },
          {
          "description": "Tageswetter für Rom am gewünschten Datum abrufen",
          "tools": [
          {
          "id": "wetter_roma",
          "name": "get_weather",
          "arguments": {
          "lat": { "$ref": "geo_roma.lat" },
          "lon": { "$ref": "geo_roma.lon" },
          "start_date": "2026-01-23",
          "end_date": "2026-01-23",
          "include_raw": false
          }
          }
          ]
          }
          ]
        - Nutzer-Eingabe:
          - Wenn ein Argument direkt aus der aktuellen Nutzernachricht kommen soll (z.B. Stadtname, Datum, Budget), verwende eine Referenz mit Präfix "user.".
          - Beispiele:
              - { "$ref": "user.origin" } → Herkunftsort aus der Nutzernachricht.
              - { "$ref": "user.destination" } → Zielort aus der Nutzernachricht.
              - { "$ref": "user.raw" } → komplette Roh-Eingabe des Nutzers.
          - Du definierst diese Pfade semantisch, damit für das Backend klar ist, welcher Teil des User-Inputs gemeint ist.
        - Zusammenfassung für $ref:
          - $ref steht IMMER als Wert eines Feldes in "arguments".
          - Du verwendest KEIN "arguments": { "$ref": ... } ohne weitere Felder.
          - Du referenzierst NIE "result", "arguments" oder ähnliche Meta-Felder, sondern arbeitest so, als ob die "result"-Struktur selbst die Basis für den Pfad ist.
        - Planung von Tool-Chains:
          - Du DARFST und SOLLST Tool-Ketten planen (z.B. "zuerst Geocoding, dann Routenberechnung, dann Bewertung").
          - Überlege explizit, welche Tools Vorbedingungen anderer Tools erfüllen (z.B. Koordinaten, IDs, Zeiträume).
          - Baue den Plan so, dass:
            - Schritt 0: Tools, die direkt aus dem User-Input und ggf. Kontext ihre Argumente beziehen.
            - Schritt 1: Tools, deren Argumente via $ref aus Ergebnissen von Schritt 0 kommen.
            - Schritt 2: Tools, die auf Schritt 1 aufbauen, usw.
          - Tools, die keine gemeinsamen Abhängigkeiten haben, dürfen im selben Schritt stehen, damit sie parallel ausgeführt werden können.
          - Du darfst mehrere Schritte vorausplanen. Das Backend kann nach Ausführung eines oder mehrerer Schritte den Plan durch einen erneuten Aufruf des Modells aktualisieren lassen.
          - Wenn du einen bestehenden Plan aktualisierst erhalten die bereits ausgeführten Schritte die selben Bezeichner wie zuvor. Toolnamen ausgeführter Tools dürfen Rückwirkend also nie verändert werden.
        - Nutzung von MCP-Tools:
          - Nutze Tools, wenn du:
            - Informationen brauchst, die nicht sicher in deinem Weltwissen oder im bereitgestellten Kontext stehen, oder
            - exakte Daten (z.B. Koordinaten, Preise, Zeiten, Verfügbarkeiten) benötigst.
          - Nutze Tools nur dann, wenn du ihre Input-Schemata sinnvoll und vollständig (ggf. mit $ref) befüllen kannst.
          - Verwende für Tool-Argumente ausschließlich:
            - explizite Informationen aus der aktuellen Nutzernachricht (per $ref "user.…"),
            - frühere Tool-Ergebnisse (per $ref auf Tool-IDs),
            - oder wohldefinierte konstante/literale Werte.
          - Du DARFST Tool-Argumente NICHT aus deinem allgemeinen Weltwissen raten.
          - Wenn die Anfrage ohne Tools gut beantwortbar ist (reine Beratung, Einschätzung, Inspiration), dann erstelle einen JSON-Output mit:
            - "steps": [] (oder nur erklärende Schritte ohne Tools) und
            - "final": "<deine natürliche Antwort>".
        - Umgang mit Wissen und Unsicherheit:
          - Erfinde keine Fakten und spekuliere nicht.
          - Wenn eine Information weder in deinem Weltwissen noch im bereitgestellten Kontext vorkommt und auch nicht per MCP-Tool-Call erlangt werden kann, erkläre dies in der "final"-Antwort.
          - Wenn dir wirklich Informationen fehlen, formuliere in "final" eine ehrliche, erklärende Antwort und beschreibe, was du stattdessen sicher sagen kannst.
        - Einschränkungen (Ablehnungsfälle im Reise-Kontext):
          - Lehne Anfragen ab, die eindeutig NICHT zum Reise-/Tourismuskontext gehören.
          - In diesem Fall gibst du ebenfalls ein JSON-Objekt zurück, z.B.:
            {
            "steps": [],
            "final": "Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht."
            }
        - Tools (verfügbare MCP-Tools):
    """
    return system_prompt + tool_catalog


In [47]:
agent_system_prompt = build_agent_system_prompt(tool_names_with_meta)

In [48]:
tool_names_with_meta = await fetch_tools_with_metadata()
from datetime import date

# Returns the current local date
today = date.today()

user_input = f"Ich möchte eine Reise nach Rom machen. Wie ist heute ({today}) das Wetter in Rom?"
tool_history = None

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=agent_system_prompt,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/25/26 08:56:14] INFO     Created new          ]8;id=316968;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=130776;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             ea9b93c94676488290ed                               
                             aeebcdcd45ec                                       
                    INFO     Processing request of type            ]8;id=413465;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=918072;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

INFO:     127.0.0.1:34230 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:34232 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:34246 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:34260 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:34274 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

    Deine Aufgaben:
    - Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
    - Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
    - Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

    Nutzung von Kontext (RAG):
    - Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für d

Jetzt vollautomatiosierung

Plan-Parsing

In [49]:
import json
from typing import Any, Dict

def agentic_parse_model_plan(output_str: str) -> Dict[str, Any]:
    print("[DEBUG] raw output:", repr(output_str))

    # Leer- oder Nonsens-Output früh abfangen
    if not output_str or not output_str.strip():
        raise ValueError("Model output is empty, cannot parse JSON plan.")

    # Die erste öffnende Klammer suchen
    start = output_str.find("{")
    if start == -1:
        raise ValueError(f"No JSON object found in model output: {output_str!r}")

    s = output_str

    # Jetzt von 'start' an die passende schließende Klammer suchen
    depth = 0
    in_string = False
    escape = False
    end: Optional[int] = None

    for i, ch in enumerate(s[start:], start=start):
        if in_string:
            # Innerhalb eines Strings
            if escape:
                # nächstes Zeichen ist escaped, einfach überspringen
                escape = False
            elif ch == "\\":
                escape = True
            elif ch == '"':
                in_string = False
        else:
            # Außerhalb von Strings
            if ch == '"':
                in_string = True
            elif ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    end = i
                    break

    if end is None:
        # Fallback: alter Modus (kannst du auch wegwerfen, wenn du möchtest)
        trimmed = s[start:]
    else:
        trimmed = s[start:end + 1]

    print("[DEBUG] trimmed JSON candidate:", trimmed)

    try:
        plan = json.loads(trimmed)
    except json.JSONDecodeError as e:
        raise ValueError(
            f"Failed to parse JSON plan from model output. "
            f"Error: {e}. Trimmed candidate: {trimmed!r}"
        ) from e

    if "steps" not in plan or "final" not in plan:
        raise ValueError("Model output must contain 'steps' and 'final'.")

    if not isinstance(plan["steps"], list):
        raise ValueError("'steps' must be a list.")

    return plan

$ref-Resolver

In [50]:
from typing import Any, Dict, Optional

def agentic_resolve_argument(
    value: Any,
    tool_results: Dict[str, Any],
    user_ctx: Dict[str, Any],
) -> Any:
    """
    Löst einzelne Argumentwerte auf. Unterstützt:
    - {"$ref": "geo_roma.lat"}
    - "$ref:geo_roma.lat"
    - "$ref:user.raw"
    """
    ref_str: Optional[str] = None

    # Fall 1: Objekt mit "$ref"
    if isinstance(value, dict) and "$ref" in value and isinstance(value["$ref"], str):
        ref_str = value["$ref"]

    # Fall 2: String mit Präfix "$ref:"
    elif isinstance(value, str) and value.startswith("$ref:"):
        ref_str = value[len("$ref:") :]

    # Kein $ref → Wert unverändert zurückgeben
    if ref_str is None:
        return value

    # Jetzt ref_str auswerten, z.B. "geo_roma.lat" oder "user.raw"
    parts = ref_str.split(".")
    root = parts[0]
    path = parts[1:]

    # Quelle: User-Kontext
    if root == "user":
        current: Any = user_ctx
    else:
        # Quelle: Tool-Resultate
        if root not in tool_results:
            raise KeyError(f"Unknown tool id in $ref: {root}")
        current = tool_results[root]

    # Debug: einmal zeigen, was wir da wirklich haben
    print(f"[DEBUG] Resolving $ref '{ref_str}': root='{root}', initial_type={type(current)}, initial_value={current}")

    for p in path:
        # Numerische Indizes unterstützen (z.B. itineraries.0)
        if isinstance(current, list) and p.isdigit():
            idx = int(p)
            current = current[idx]
        elif isinstance(current, dict) and p in current:
            current = current[p]
        else:
            # Noch mehr Debug, bevor wir crashen
            raise KeyError(
                f"Invalid path component '{p}' in $ref '{ref_str}' "
                f"(current_type={type(current)}, current_value={current})"
            )

    return current


def agentic_resolve_tool_arguments(arguments, tool_results, user_ctx):
    if isinstance(arguments, dict):
        # Spezialfall: dict besteht NUR aus "$ref" → ersetze das ganze Dict durch das referenzierte Objekt
        if set(arguments.keys()) == {"$ref"} and isinstance(arguments["$ref"], str):
            return agentic_resolve_argument(arguments, tool_results, user_ctx)

        return {
            k: agentic_resolve_tool_arguments(v, tool_results, user_ctx)
            for k, v in arguments.items()
        }
    elif isinstance(arguments, list):
        return [
            agentic_resolve_tool_arguments(v, tool_results, user_ctx)
            for v in arguments
        ]
    else:
        return agentic_resolve_argument(arguments, tool_results, user_ctx)

Einen Step ausführen

In [51]:
from typing import Callable, Awaitable

async def agentic_execute_step(
    step: Dict[str, Any],
    call_tool_fn: Callable[[str, Dict[str, Any]], Awaitable[Any]],
    tool_results: Dict[str, Any],
    user_ctx: Dict[str, Any],
) -> List[Dict[str, Any]]:
    """
    Führt alle Tools in einem Step aus.
    - step: {"description": "...", "tools": [ {id, name, arguments}, ... ]}
    - call_tool_fn: deine MCP-Invoker-Funktion (z.B. call_mcp_tool)
    - tool_results: wird in-place mit neuen Ergebnissen gefüllt
    - user_ctx: z.B. {"raw": user_input}

    Rückgabe: Liste von History-Objekten für den Prompt (z.B. Tool-Logs).
    """
    history_entries: List[Dict[str, Any]] = []

    for tool in step.get("tools", []):
        tool_id = tool["id"]
        tool_name = tool["name"]
        raw_args = tool.get("arguments", {})

        # $ref auflösen
        resolved_args = agentic_resolve_tool_arguments(raw_args, tool_results, user_ctx)

        # Tool aufrufen
        print(f"[{tool_id}] {tool_name} resolved arguments: {resolved_args}")
        result = await call_tool_fn(tool_name, resolved_args)
        print(f"toolcallresult: {result}")

        tool_id = tool.get("id")

        # MCP CallToolResult → direkt structuredContent speichern
        if hasattr(result, "structuredContent") and result.structuredContent is not None:
            normalized_result = result.structuredContent
        # Falls du irgendwo schon dicts baust, die structuredContent enthalten
        elif isinstance(result, dict) and "structuredContent" in result and result["structuredContent"] is not None:
            normalized_result = result["structuredContent"]
        else:
            normalized_result = result

        print(f"[DEBUG] Stored tool_result[{tool_id}] = {type(normalized_result)} → {normalized_result}")

        # History-Eintrag für Tools (kannst du an dein Format anpassen)

        if normalized_result.get("isError"):
            raise Exception(f"Tool failed with error: {normalized_result.get('isError')}")

        if isinstance(normalized_result, dict) and "data" in normalized_result:
            res = normalized_result["data"]
        else:
            res = normalized_result

        # Ergebnis speichern für for Schleife bei parallelen Tools
        tool_results[tool_id] = res

        history_entries.append({
            "role": "tool",
            "tool_id": tool_id,
            "name": tool_name,
            "arguments": resolved_args,
            "result": res,
        })

    return history_entries

Nächsten Step wählen

In [52]:
from typing import Set

def agentic_find_next_step(
    plan: Dict[str, Any],
    executed_step_indices: Set[int],
) -> Optional[int]:
    """
    Wählt den nächsten noch nicht ausgeführten Step.
    V1: simpel – der kleinste Index, der noch nicht in executed_step_indices ist.
    """
    for idx, _ in enumerate(plan.get("steps", [])):
        if idx not in executed_step_indices:
            return idx
    return None

Prompt-Building (Rename + leichte Erweiterung)

In [53]:
def agentic_build_chat_prompt_with_rag_and_tools(
    system_prompt: str,
    user_prompt: str,
    history: List[Dict[str, Any]],
    tool_history: Optional[List[Dict[str, Any]]] = None,
    retrieved_context: Optional[str] = None,
) -> str:
    """
    Baut den vollständigen Prompt für das LLM, inkl.:
    - system_prompt
    - optionaler Kontext (RAG)
    - bisherige Chat-History
    - bisherige Tool-History
    - aktuelle User-Nachricht
    """
    parts = []
    parts.append("<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n")
    parts.append(system_prompt)
    parts.append("<|eot_id|>")

    if retrieved_context:
        parts.append("<|start_header_id|>system<|end_header_id|>\n\n")
        parts.append(f"Kontext (aus Retrieval):\n{retrieved_context}\n")
        parts.append("<|eot_id|>")

    # alte history anhängen
    if history:
        for msg in history:
            parts.append(f"<|start_header_id|>{msg['role']}<|end_header_id|>\n\n")
            parts.append(msg["content"])
            parts.append("<|eot_id|>")

    # tool_history (falls du sie als eigene „role“ einfügst)
    if tool_history:
        for th in tool_history:
            parts.append(f"<|start_header_id|>tool<|end_header_id|>\n\n")
            parts.append(json.dumps(th, ensure_ascii=False))
            parts.append("<|eot_id|>")

    # Aktuelle Userfrage
    parts.append("<|start_header_id|>user<|end_header_id|>\n\n")
    parts.append(user_prompt)
    parts.append("<|eot_id|>")

    return "".join(parts)

Orchestrator: Alles zusammen

In [54]:
async def agentic_run_to_final_answer(
    user_input: str,
    system_prompt: str,
    history: List[Dict[str, Any]],
    tool_history: Optional[List[Dict[str, Any]]],
    call_tool_fn: Callable[[str, Dict[str, Any]], Awaitable[Any]],
    build_prompt_fn: Callable[..., str] = agentic_build_chat_prompt_with_rag_and_tools,
    max_iterations: int = 8,
) -> str:
    """
    Hauptschleife:
    - plant mit dem LLM,
    - führt Step für Step Tools aus,
    - replanned,
    - liefert am Ende plan['final'] zurück.
    """
    if tool_history is None:
        tool_history = []

    tool_results: Dict[str, Any] = {}

    executed_step_indices: Set[int] = set()

    # User-Kontext für $ref: "user.*"
    user_ctx = {
        "raw": user_input,
        # hier könntest du später origin/destination/etc. extrahieren
    }

    for iteration in range(max_iterations):
        # 1) Prompt bauen + Plan vom Modell holen
        prompt = build_prompt_fn(
            system_prompt=system_prompt,
            user_prompt=user_input,
            history=history,
            tool_history=tool_history,
        )

        print(f"iteration: {iteration}, prompt: {prompt!r}")
        assistant_text = llama_chat(prompt, max_new_tokens=1024)
        print(f"iteration: {iteration}, assistant_text: {assistant_text!r}")

        # 2) Plan parsen
        plan = agentic_parse_model_plan(assistant_text)
        print(f"iteration: {iteration}, plan: {plan!r}")

        # 3) Wenn final schon vorhanden und keine offenen Steps → fertig
        next_idx = agentic_find_next_step(plan, executed_step_indices)
        print(f"iteration: {iteration}, next_idx: {next_idx!r}")

        if plan.get("final") is not None and next_idx is None:
            # finale Antwort
            final = plan["final"]
            print(f"iteration: {iteration}, final: {final}")
            return plan["final"]

        # 4) Falls es einen nächsten Step gibt → ausführen
        if next_idx is not None:
            step = plan["steps"][next_idx]
            step_history_entries = await agentic_execute_step(
                step=step,
                call_tool_fn=call_tool_fn,
                tool_results=tool_results,
                user_ctx=user_ctx,
            )
            executed_step_indices.add(next_idx)

            # Tool-Historie erweitern (für nächsten Prompt)
            tool_history.extend(step_history_entries)

            # Nach jedem Step neue Iteration starten → replanning möglich
            continue

        # 5) Kein Step mehr, aber final ist noch None → Modell „nötigen“, final zu setzen
        if plan.get("final") is None and next_idx is None:
            # Einfach eine weitere Runde, diesmal mit Hinweis im User- oder Systemprompt,
            # dass jetzt eine finale Antwort formuliert werden soll.
            user_input = "Formuliere jetzt bitte eine finale Antwort für den Nutzer basierend auf allen bisherigen Tool-Ergebnissen."
            continue

    # Fallback, falls max_iterations erreicht
    return "Ich konnte trotz mehrerer Planungsrunden keine finale Antwort erzeugen."

olcall per Invoke

In [55]:
from mcp.types import TextContent  # ggf. anpassen

async def agentic_call_mcp_tool(tool_name: str, args: dict) -> dict:
    print(f"[DEBUG] agentic_call_mcp_tool: {tool_name}({args})")
    async with streamable_http_client(MCP_URL) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            raw_result = await session.call_tool(tool_name, args)
            print(f"[DEBUG] Raw tool result type: {type(raw_result)}")
            print(f"[DEBUG] Raw tool result repr: {repr(raw_result)[:400]}")

            # Normalisieren
            result_dict: dict = {
                "isError": getattr(raw_result, "isError", False),
            }

            # strukturierte Inhalte bevorzugen
            if getattr(raw_result, "structuredContent", None) is not None:
                result_dict["data"] = raw_result.structuredContent
            else:
                # Fallback: Text aus content zusammenbauen
                texts = []
                for c in getattr(raw_result, "content", []) or []:
                    if isinstance(c, TextContent):
                        texts.append(c.text)
                result_dict["data"] = "\n".join(texts)

            return result_dict

Test

In [57]:
history_agentic = [
    {
        "role": "user",
        "content": "Hi, mein Name ist Max Mustermann. Sprich mich bitte in jeder Antwort mit meinem Namen an. Sag explizit dazu, dass du mich in jeder Antwort mit meinem Namen ansprechen wirst."
    },
    # weitere Messages:
    # {"role": "assistant", "content": "..."},
]

user_input = f"Ich möchte eine Reise nach Barcelona machen. Wie ist heute ({today}) das Wetter in Barcelona?"

final_answer = await agentic_run_to_final_answer(
    user_input=user_input,
    system_prompt=agent_system_prompt,
    history=history_agentic,
    tool_history=None,
    call_tool_fn=agentic_call_mcp_tool,          # deine bestehende MCP-Tool-Invoker-Funktion
    build_prompt_fn=agentic_build_chat_prompt_with_rag_and_tools,
)

print("Finale Antwort an den User:")
print(final_answer)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


iteration: 0, prompt: '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n\n    Du bist ein persönlicher Reiseplaner.\n\n    Deine Aufgaben:\n    - Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.\n    - Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).\n    - Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.\n\n    Nutzung von Kontext (RAG):\n    - Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.\n    - Du darfst alle darin enthaltenen Fakten verwenden.\n    - Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.\n\n    Allgemeines Arbeitsprinzip (ReAct mit Planung und Tool-Chains):\n    - Denke zuerst still über ei

[01/25/26 09:18:42] INFO     Created new          ]8;id=52776;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=456825;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             9e3699005f03486b800a                               
                             4be0101bec8c                                       
                    INFO     Processing request of type            ]8;id=11331;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=841601;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py#

iteration: 0, assistant_text: 'assistant\n\n{\n  "steps": [\n    {\n      "description": "Geokoordinaten für Barcelona bestimmen",\n      "tools": [\n        {\n          "id": "geo_barcelona",\n          "name": "geocode",\n          "arguments": {\n            "destination": "Barcelona"\n          }\n        }\n      ]\n    },\n    {\n      "description": "Wetter für Barcelona am 25.01.2026 abrufen",\n      "tools": [\n        {\n          "id": "wetter_barcelona",\n          "name": "get_weather",\n          "arguments": {\n            "lat": { "$ref": "geo_barcelona.lat" },\n            "lon": { "$ref": "geo_barcelona.lon" },\n            "start_date": "2026-01-25",\n            "end_date": "2026-01-25",\n            "include_raw": false\n          }\n        }\n      ]\n    }\n  ],\n  "final": null\n}'
[DEBUG] raw output: 'assistant\n\n{\n  "steps": [\n    {\n      "description": "Geokoordinaten für Barcelona bestimmen",\n      "tools": [\n        {\n          "id": "geo_barcelona

ValueError: Model output must contain 'steps' and 'final'.

In [ ]:
history_agentic = [
    {
        "role": "user",
        "content": "Hi, mein Name ist Max Mustermann. Sprich mich bitte in jeder Antwort mit meinem Namen an. Sag explizit dazu, dass du mich in jeder Antwort mit meinem Namen ansprechen wirst."
    },
    # weitere Messages:
    # {"role": "assistant", "content": "..."},
]

user_input = f"Was sind die besten Plätze im Umkreis von 5km rund um Barcelona um zu joggen?"

final_answer = await agentic_run_to_final_answer(
    user_input=user_input,
    system_prompt=agent_system_prompt,
    history=history_agentic,
    tool_history=None,
    call_tool_fn=agentic_call_mcp_tool,          # deine bestehende MCP-Tool-Invoker-Funktion
    build_prompt_fn=agentic_build_chat_prompt_with_rag_and_tools,
)

print("Finale Antwort an den User:")
print(final_answer)

In [ ]:
history_agentic = [
    {
        "role": "user",
        "content": "Hi, mein Name ist Max Mustermann. Sprich mich bitte in jeder Antwort mit meinem Namen an. Sag explizit dazu, dass du mich in jeder Antwort mit meinem Namen ansprechen wirst."
    },
    # weitere Messages:
    # {"role": "assistant", "content": "..."},
]

user_input = f"Ich möchte ein Razepato in Barcelona sehen. Wann und wohin muss ich reisen?"

final_answer = await agentic_run_to_final_answer(
    user_input=user_input,
    system_prompt=agent_system_prompt,
    history=history_agentic,
    tool_history=None,
    call_tool_fn=agentic_call_mcp_tool,          # deine bestehende MCP-Tool-Invoker-Funktion
    build_prompt_fn=agentic_build_chat_prompt_with_rag_and_tools,
)

print("Finale Antwort an den User:")
print(final_answer)